# Introduction to Bedrock - Fine-Tuning

> *If you see errors, you may need to be allow-listed for the Bedrock models used by this notebook*

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*


In this demo notebook, we demonstrate how to use the Bedrock Python SDK for fine-tuning Bedrock models with your own data. If you have text samples to train and want to adapt the Bedrock models to your domain, you can further fine-tune the Bedrock foundation models by providing your own training datasets. You can upload your datasets to Amazon S3, and provide the S3 bucket path while configuring a Bedrock fine-tuning job. You can also adjust hyper parameters (learning rate, epoch, and batch size) for fine-tuning. After the fine-tuning job of the model with your dataset has completed, you can start using the model for inference in the Bedrock playground application. You can select the fine-tuned model and submit a prompt to the fine-tuned model along with a set of model parameters. The fine-tuned model should generate texts to be more alike your text samples. 

-----------

1. Setup
2. Fine-tuning
3. Testing the fine-tuned model

 Note: This notebook was tested in Amazon SageMaker Studio with Python 3 (Data Science 2.0) kernel.

---

## 1. Setup

In [2]:
%pip install -U boto3 botocore --force-reinstall --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
awscli 1.29.63 requires botocore==1.31.63, but you have botocore 1.31.70 which is incompatible.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.3 which is incompatible.
jupyterlab 3.4.4 requires jupyter-server~=1.16, but you have jupyter-server 2.7.3 which is incompatible.
jupyterlab-server 2.10.3 requires jupyter-server~=1.4, but you have jupyter-server 2.7.3 which is incompatible.
notebook 6.5.6 requires jupyter-client<8,>=5.3.4, but you have jupyter-client 8.4.0 which is incompatible.
notebook 6.5.6 requires pyzmq<25,>=17, but you have pyzmq 25.1.1 which is incompatible.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.3.0 which is incompatible.
s

In [3]:
%pip list | grep boto3

boto3                                1.28.70

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


#### Now let's set up our connection to the Amazon Bedrock SDK using Boto3

In [4]:
#### Un comment the following lines to run from your local environment outside of the AWS account with Bedrock access

#import os
#os.environ['BEDROCK_ASSUME_ROLE'] = '<YOUR_VALUES>'
#os.environ['AWS_PROFILE'] = '<YOUR_VALUES>'

In [5]:
import boto3
import json
import os
import sys

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

bedrock_client = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=False  # Needed for control plane
)

Create new client
  Using region: us-west-2
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-west-2.amazonaws.com)


In [ ]:
bedrock_client.list_foundation_models()

In [7]:
import sagemaker

sess = sagemaker.Session()
sagemaker_session_bucket = sess.default_bucket()
role = sagemaker.get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


### Preview custom data

Our custom data in JSON line file format.

In [8]:
data = "data/train.jsonl"

Read the JSON line file into an object like any normal file

In [9]:
with open(data) as f:
    lines = f.read().splitlines()

#### Load the ‘lines’ object into a pandas Data Frame.

In [10]:
import pandas as pd
df_inter = pd.DataFrame(lines)
df_inter.columns = ['json_element']

This intermediate data frame will have only one column with each json object in a row. A sample output is given below.

In [11]:
df_inter['json_element'].apply(json.loads)

0     {'output': 'Positive', 'input': 'the rock is d...
1     {'output': 'Positive', 'input': 'the gorgeousl...
2     {'output': 'Positive', 'input': 'effective but...
3     {'output': 'Positive', 'input': 'if you someti...
4     {'output': 'Positive', 'input': 'emerges as so...
5     {'output': 'Positive', 'input': 'the film prov...
6     {'output': 'Positive', 'input': 'offers that r...
7     {'output': 'Positive', 'input': 'perhaps no pi...
8     {'output': 'Positive', 'input': 'steers turns ...
9     {'output': 'Positive', 'input': 'take care of ...
10    {'output': 'Negative', 'input': 'no worse than...
11    {'output': 'Negative', 'input': 'the plot is s...
12    {'output': 'Negative', 'input': 'at first , th...
13    {'output': 'Negative', 'input': 'never again s...
14    {'output': 'Negative', 'input': 'the story its...
15    {'output': 'Negative', 'input': 'technically ,...
16    {'output': 'Negative', 'input': 'the title's l...
17    {'output': 'Negative', 'input': 'the parts

Now we will apply json loads function on each row of the ‘json_element’ column. ‘json.loads’ is a decoder function in python which is used to decode a json object into a dictionary. ‘apply’ is a popular function in pandas that takes any function and applies to each row of the pandas dataframe or series.

In [12]:
df_final = pd.json_normalize(df_inter['json_element'].apply(json.loads))

Once decoding is done we will apply the json normalize function to the above result. json normalize will convert any semi-structured json data into a flat table. Here it converts the JSON ‘keys’ to columns and its corresponding values to row elements.

In [13]:
df_final

,output,input
0,Positive,the rock is destined to be the 21st century's ...
1,Positive,the gorgeously elaborate continuation of the l...
2,Positive,effective but too-tepid biopic
3,Positive,if you sometimes like to go to the movies to h...
4,Positive,"emerges as something rare , an issue movie tha..."
5,Positive,the film provides some great insight into the ...
6,Positive,offers that rare combination of entertainment ...
7,Positive,perhaps no picture ever made has more literall...
8,Positive,steers turns in a snappy screenplay that curls...
9,Positive,take care of my cat offers a refreshingly diff...


### Uploading data to S3

Next, we need to upload our training dataset to S3:

In [14]:
s3_location = f"s3://{sagemaker_session_bucket}/bedrock/finetuning/train.jsonl"
s3_output = f"s3://{sagemaker_session_bucket}/bedrock/finetuning/output"

In [15]:
!aws s3 cp data/train.jsonl $s3_location

upload: data/train.jsonl to s3://sagemaker-us-west-2-079002598131/bedrock/finetuning/train.jsonl


Now we can create the fine-tuning job. 

### ^^ **Note:** Make sure the IAM role you're using has these [IAM policies](https://docs.aws.amazon.com/bedrock/latest/userguide/model-customization-iam-role.html) attached that allow Amazon Bedrock access to the specified S3 buckets ^^

## 2. Fine-tuning

In [16]:
import time
timestamp = int(time.time())

In [21]:
base_model_id = "amazon.titan-text-express-v1"
#base_model_id = "amazon.titan-text-lite-v1"

In [22]:
job_name = "titan-{}".format(timestamp)
job_name

'titan-1698257909'

In [23]:
custom_model_name = "custom-{}".format(job_name)
custom_model_name

'custom-titan-1698257909'

In [24]:
bedrock_client.create_model_customization_job(
    jobName=job_name,
    customModelName=custom_model_name,
    roleArn=role,
    baseModelIdentifier=base_model_id,
    hyperParameters = {
        "epochCount": "1",
        "batchSize": "1",
        "learningRate": "0.005",
        "learningRateWarmupSteps": "0"
    },
    trainingDataConfig={"s3Uri": s3_location},
    outputDataConfig={"s3Uri": s3_output},
)

{'ResponseMetadata': {'RequestId': 'dd128903-efdf-4d5a-8f52-41c3d80ae065',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Wed, 25 Oct 2023 18:20:20 GMT',
   'content-type': 'application/json',
   'content-length': '122',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'dd128903-efdf-4d5a-8f52-41c3d80ae065'},
  'RetryAttempts': 0},
 'jobArn': 'arn:aws:bedrock:us-west-2:079002598131:model-customization-job/amazon.titan-text-express-v1:0:8k/ycbprb70zy42'}

In [25]:
status = bedrock_client.get_model_customization_job(jobIdentifier=job_name)["status"]
status

'InProgress'

# Let's periodically check in on the progress.
### The next cell might run for ~40min

In [26]:
import time

status = bedrock_client.get_model_customization_job(jobIdentifier=job_name)["status"]

while status == "InProgress":
    print(status)
    time.sleep(30)
    status = bedrock_client.get_model_customization_job(jobIdentifier=job_name)["status"]
    
print(status)

InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress

In [27]:
completed_job = bedrock_client.get_model_customization_job(jobIdentifier=job_name)
completed_job

{'ResponseMetadata': {'RequestId': '87c0673f-969c-458d-b413-9b95ae61ed42',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 25 Oct 2023 19:57:12 GMT',
   'content-type': 'application/json',
   'content-length': '1209',
   'connection': 'keep-alive',
   'x-amzn-requestid': '87c0673f-969c-458d-b413-9b95ae61ed42'},
  'RetryAttempts': 0},
 'jobArn': 'arn:aws:bedrock:us-west-2:079002598131:model-customization-job/amazon.titan-text-express-v1:0:8k/ycbprb70zy42',
 'jobName': 'titan-1698257909',
 'outputModelName': 'custom-titan-1698257909',
 'outputModelArn': 'arn:aws:bedrock:us-west-2:079002598131:custom-model/amazon.titan-text-express-v1:0:8k/a3ne5s6g0xc4',
 'clientRequestToken': 'c6f03e80-2903-4e34-b7a9-f7584415047a',
 'roleArn': 'arn:aws:iam::079002598131:role/service-role/AmazonSageMaker-ExecutionRole-20220804T150518',
 'status': 'Completed',
 'creationTime': datetime.datetime(2023, 10, 25, 18, 20, 20, 815000, tzinfo=tzlocal()),
 'lastModifiedTime': datetime.datetime(2023, 10, 25

## 3. Testing

Now we can test the fine-tuned model

In [28]:
bedrock_client.list_custom_models()

{'ResponseMetadata': {'RequestId': 'f8dc45ac-f08d-41ca-ada8-943a1257b474',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 25 Oct 2023 19:57:12 GMT',
   'content-type': 'application/json',
   'content-length': '657',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'f8dc45ac-f08d-41ca-ada8-943a1257b474'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-west-2:079002598131:custom-model/amazon.titan-text-express-v1:0:8k/a3ne5s6g0xc4',
   'modelName': 'custom-titan-1698257909',
   'creationTime': datetime.datetime(2023, 10, 25, 18, 20, 20, 815000, tzinfo=tzlocal()),
   'baseModelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-text-express-v1:0:8k',
   'baseModelName': ''},
  {'modelArn': 'arn:aws:bedrock:us-west-2:079002598131:custom-model/amazon.titan-text-express-v1:0:8k/rfs7atdiuu9m',
   'modelName': 'my-finetuned-model-small-10',
   'creationTime': datetime.datetime(2023, 10, 14, 15, 56, 53, 673000, tzinfo=tzlocal()),
   'baseM

In [29]:
for job in bedrock_client.list_model_customization_jobs()["modelCustomizationJobSummaries"]:
    print("-----\n" + "jobArn: " + job["jobArn"] + "\njobName: " + job["jobName"] + "\nstatus: " + job["status"] + "\ncustomModelName: " + job["customModelName"])

-----
jobArn: arn:aws:bedrock:us-west-2:079002598131:model-customization-job/amazon.titan-text-express-v1:0:8k/ycbprb70zy42
jobName: titan-1698257909
status: Completed
customModelName: custom-titan-1698257909
-----
jobArn: arn:aws:bedrock:us-west-2:079002598131:model-customization-job/amazon.titan-text-express-v1:0:8k/qogsvgm6j43o
jobName: finetune-small-10
status: Completed
customModelName: my-finetuned-model-small-10


## GetCustomModel

In [30]:
bedrock_client.get_custom_model(modelIdentifier=custom_model_name)

{'ResponseMetadata': {'RequestId': 'a5c7fef6-669d-491d-bccf-477e23938a52',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 25 Oct 2023 19:57:13 GMT',
   'content-type': 'application/json',
   'content-length': '866',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'a5c7fef6-669d-491d-bccf-477e23938a52'},
  'RetryAttempts': 0},
 'modelArn': 'arn:aws:bedrock:us-west-2:079002598131:custom-model/amazon.titan-text-express-v1:0:8k/a3ne5s6g0xc4',
 'modelName': 'custom-titan-1698257909',
 'jobArn': 'arn:aws:bedrock:us-west-2:079002598131:model-customization-job/amazon.titan-text-express-v1:0:8k/ycbprb70zy42',
 'baseModelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-text-express-v1:0:8k',
 'hyperParameters': {'batchSize': '1',
  'epochCount': '1',
  'learningRate': '0.005',
  'learningRateWarmupSteps': '0'},
 'trainingDataConfig': {'s3Uri': 's3://sagemaker-us-west-2-079002598131/bedrock/finetuning/train.jsonl'},
 'outputDataConfig': {'s3Uri': 's3://sagemaker-us-

In [31]:
custom_model_arn = bedrock_client.get_custom_model(modelIdentifier=custom_model_name)['modelArn']
custom_model_arn

'arn:aws:bedrock:us-west-2:079002598131:custom-model/amazon.titan-text-express-v1:0:8k/a3ne5s6g0xc4'

In [32]:
base_model_arn = bedrock_client.get_custom_model(modelIdentifier=custom_model_name)['baseModelArn']
base_model_arn

'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-text-express-v1:0:8k'

## **Note:** To invoke custom models, you need to first create a provisioned throughput resource and make requests using that resource.

In [ ]:
provisioned_model_name = "{}-provisioned".format(custom_model_name)
provisioned_model_name

## !! **Note:** SDK currently only supports 1 month and 6 months commitment terms. Go to Bedrock console to manually purchase no commitment term option for testing !!

In [ ]:
# bedrock_client.create_provisioned_model_throughput(
#     modelUnits = 1,
#     commitmentDuration = "OneMonth", ## Note: SDK is currently missing No Commitment option
#     provisionedModelName = provisioned_model_name,
#     modelId = base_model_arn
# ) 

## ListProvisionedModelThroughputs

In [33]:
bedrock_client.list_provisioned_model_throughputs()["provisionedModelSummaries"]

[{'provisionedModelName': 'custom-titan-1698257909-provisioned',
  'provisionedModelArn': 'arn:aws:bedrock:us-west-2:079002598131:provisioned-model/fas0m0hl45m0',
  'modelArn': 'arn:aws:bedrock:us-west-2:079002598131:custom-model/amazon.titan-text-express-v1:0:8k/a3ne5s6g0xc4',
  'desiredModelArn': 'arn:aws:bedrock:us-west-2:079002598131:custom-model/amazon.titan-text-express-v1:0:8k/a3ne5s6g0xc4',
  'foundationModelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-text-express-v1:0:8k',
  'modelUnits': 0,
  'desiredModelUnits': 1,
  'status': 'Creating',
  'creationTime': datetime.datetime(2023, 10, 25, 20, 1, 3, 688000, tzinfo=tzlocal()),
  'lastModifiedTime': datetime.datetime(2023, 10, 25, 20, 1, 29, 333000, tzinfo=tzlocal())}]

## GetProvisionedModelThroughput

In [34]:
#provisioned_model_name = "<YOUR_PROVISIONED_MODEL_NAME>" # e.g. custom-titan-1698257909-provisioned
provisioned_model_name = "custom-titan-1698257909-provisioned" 

In [35]:
provisioned_model_arn = bedrock_client.get_provisioned_model_throughput(
     provisionedModelId=provisioned_model_name)["provisionedModelArn"]
provisioned_model_arn

'arn:aws:bedrock:us-west-2:079002598131:provisioned-model/fas0m0hl45m0'

In [36]:
deployment_status = bedrock_client.get_provisioned_model_throughput(
    provisionedModelId=provisioned_model_name)["status"]
deployment_status

'Creating'

## The next cell might run for ~10min

In [ ]:
import time

deployment_status = bedrock_client.get_provisioned_model_throughput(
    provisionedModelId=provisioned_model_name)["status"]

while deployment_status == "Creating":
    
    print(deployment_status)
    time.sleep(30)
    deployment_status = bedrock_client.get_provisioned_model_throughput(
        provisionedModelId=provisioned_model_name)["status"]  
    
print(deployment_status)

Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
InService


### Invoke Model

In [38]:
import boto3
import json
import os
import sys

# module_path = ".."
# sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

bedrock_client = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=True  # Default. Needed for invoke_model() on the data plane
)

Create new client
  Using region: us-west-2
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)


In [59]:
response = bedrock_client.invoke_model(
    # modelId needs to be Provisioned Throughput Model ARN
    modelId=provisioned_model_arn,
    body="""
{
  "inputText": "Classify this as Positive, Neutral, or Negative:\\n'I really don't like this!'",
  "textGenerationConfig":{
    "maxTokenCount": 50, 
    "stopSequences": [],
    "temperature": 1,
    "topP": 0.9
  }
}
"""
)

response_body = response["body"].read().decode('utf8')
print(response_body)

print(json.loads(response_body)["results"][0]["outputText"])

{"inputTextTokenCount":20,"results":[{"tokenCount":3,"outputText":"\nNegative","completionReason":"FINISH"}]}

Negative


# Qualitative Results with Zero Shot Inference AFTER Fine-Tuning

As with many GenAI applications, a qualitative approach where you ask yourself the question "is my model behaving the way it is supposed to?" is usually a good starting point. In the example below (the same one we started this notebook with), you can see how the fine-tuned model is able to create a reasonable summary of the dialogue compared to the original inability to understand what is being asked of the model.

In [ ]:
response = bedrock_client.invoke_model(
    # modelId needs to be Provisioned Throughput Model ARN
    modelId=provisioned_model_arn,
    body="""
{"inputText": "Classify this as Positive, Neutral, or Negative:\\n'I really don't like this!'",
 "textGenerationConfig":{
  "maxTokenCount":50,
  "stopSequences":[],
  "temperature":1,
  "topP":0.9
 }
}
""")

response_body = response["body"].read().decode('utf8')
print(response_body)

print(json.loads(response_body)["results"][0]["outputText"])

In [15]:
# idx = 2
# diag = tokenizer.decode(tokenized_dataset['test'][idx]['input_ids'], skip_special_tokens=True)
# model_input = tokenizer(diag, return_tensors="pt").input_ids
# summary = tokenizer.decode(tokenized_dataset['test'][idx]['labels'], skip_special_tokens=True)

# original_outputs = model.to('cpu').generate(
#     model_input,
#     GenerationConfig(max_new_tokens=200, num_beams=1),
# )
# outputs = tuned_model.to('cpu').generate(
#     model_input,
#     GenerationConfig(max_new_tokens=200, num_beams=1,),
# )
# text_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

# diag_print = diag.replace(' #',' \n#')
# print(f"Prompt:\n--------------------------\n{diag_print}\n--------------------------")
# print(f'Flan-T5 response: {original_text_output}')
# print(f'Our instruct-tuned response (on top of Flan-T5): {text_output}')
# print(f'Baseline summary from original dataset: {summary}')

Prompt:
--------------------------
Summarize the following conversation. 
#Person1#: Hello. My name is John Sandals, and I've got a reservation. 
#Person2#: May I see some identification, sir, please? 
#Person1#: Sure. Here you are. 
#Person2#: Thank you so much. Have you got a credit card, Mr. Sandals? 
#Person1#: I sure do. How about American Express? 
#Person2#: Unfortunately, at the present time we take only MasterCard or VISA. 
#Person1#: No American Express? Okay, here's my VISA. 
#Person2#: Thank you, sir. You'll be in room 507, nonsmoking, with a queen-size bed. Do you approve, sir? 
#Person1#: Yeah, that'll be fine. 
#Person2#: That's great. This is your key, sir. If you need anything at all, anytime, just dial zero. Summary: 
--------------------------
Flan-T5 response: John Sandals has a reservation for a room at the Venetian Hotel in Las Vegas.
Our instruct-tuned response (on top of Flan-T5): John Sandals has a reservation and checks in with his VISA. #Person2# helps him to

# Quantitative Results with ROUGE Metric

The [ROUGE metric](https://en.wikipedia.org/wiki/ROUGE_(metric)) helps quantify the validity of summarizations produced by models. It compares summarizations to a "baseline" summary which is usually created by a human. While not perfect, it does give an indication to the overall increase in summarization effectiveness that we have accomplished by fine-tuning.

In [16]:
rouge = evaluate.load('rouge')

## Evaluate a Subsection of Summaries

In [17]:
# again, for the sake of time, we will only be generating a few summaries with each model
# outside of the lab, a good exercise is to increase the number of validation summaries generated

# TODO:  what is the tokenized

#dialogues = tokenized_dataset['test'][0:10]['input_ids']
#baseline_summaries = tokenized_dataset['test'][0:10]['labels']

dialogues = dataset

# decode the original summaries
human_baseline_summaries = []
for base_summary in baseline_summaries:
    human_baseline_summaries.append(tokenizer.decode(base_summary, skip_special_tokens=True))

# generate the summaries
original_outputs = model.generate(dialogues, GenerationConfig(max_new_tokens=200))
tuned_outputs = tuned_model.generate(dialogues, GenerationConfig(max_new_tokens=200))

In [18]:
# store the summaries in lists
original_model_summaries = []
tuned_model_summaries = []

# decode all the summaries
for original_summary, tuned_summary in zip(original_outputs, tuned_outputs):
    original_model_summaries.append(tokenizer.decode(original_summary, skip_special_tokens=True))
    tuned_model_summaries.append(tokenizer.decode(tuned_summary, skip_special_tokens=True))

In [19]:
original_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries,
    use_aggregator=True,
    use_stemmer=True,
)

In [20]:
tuned_results = rouge.compute(
    predictions=tuned_model_summaries,
    references=human_baseline_summaries,
    use_aggregator=True,
    use_stemmer=True,
)

In [21]:
original_results

{'rouge1': 0.23899693678641046,
 'rouge2': 0.08932806324110672,
 'rougeL': 0.21776705653021444,
 'rougeLsum': 0.21325132275132275}

In [22]:
tuned_results

{'rouge1': 0.5243400833160587,
 'rouge2': 0.240785255433749,
 'rougeL': 0.3960164115550142,
 'rougeLsum': 0.396040090323604}

## Evalute the Full Dataset

The file called "diag-summary-training-results.csv" contains a pre-populated list of all model results which we can use to evaluate on a larger section of data. The results show substantial improvement in all ROUGE metrics!

In [23]:
import pandas as pd
results = pd.read_csv("diag-summary-training-results.csv")
original_model_summaries = results['original_model_summaries'].values
tuned_model_summaries = results['tuned_model_summaries'].values
human_baseline_summaries = results['human_baseline_summaries'].values

In [24]:
original_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

In [25]:
tuned_results = rouge.compute(
    predictions=tuned_model_summaries,
    references=human_baseline_summaries[0:len(tuned_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

In [26]:
original_results

{'rouge1': 0.2334158581572823,
 'rouge2': 0.07603964187010573,
 'rougeL': 0.20145520923859048,
 'rougeLsum': 0.20145899339006135}

In [27]:
tuned_results

{'rouge1': 0.42161291557556113,
 'rouge2': 0.18035380596301792,
 'rougeL': 0.3384439349963909,
 'rougeLsum': 0.33835653595561666}

In [28]:
improvement = (np.array(list(tuned_results.values())) - np.array(list(original_results.values())))
for key, value in zip(tuned_results.keys(), improvement):
    print(f'{key} absolute percentage improvement after instruct fine-tuning: {value*100:.2f}%')

rouge1 absolute percentage improvement after instruct fine-tuning: 18.82%
rouge2 absolute percentage improvement after instruct fine-tuning: 10.43%
rougeL absolute percentage improvement after instruct fine-tuning: 13.70%
rougeLsum absolute percentage improvement after instruct fine-tuning: 13.69%


## Delete Provisioned Throughput

When you're done testing, you can delete Provisioned Throughput to stop charges

In [ ]:
# bedrock_client = bedrock.get_bedrock_client(
#     assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
#     region=os.environ.get("AWS_DEFAULT_REGION", None),
#     runtime=False  # Default. Needed for invoke_model() on the data plane
# )

In [ ]:
# bedrock_client.delete_provisioned_model_throughput(
#     provisionedModelId = provisioned_model_name
# )